**Problem3: Monte Carlo Tree Search**
>AI20BTECH11010-Haritha

In [ ]:
#import necessary libraries
import numpy as np
from tqdm import tqdm
import random


**A)**

In [ ]:
def Num_positions_empty(state): # takes a state of the Tic-tac-toe board
    num_empty=(state==0).sum()
    return num_empty  #returns number of empty positions available in that state


In [ ]:
class Tic_Tac_Toe_Board:
    def __init__(self,state,player,parent):
        self.state=state #current state
        self.player=player  #which player
        self.n=0  #Number of visits to the state
        self.q=0  #action -value of the state
        self.w=0 #wins count
        self.l=0  #losses count
        self.childs=Num_positions_empty(self.state) #Number of empty positions of that state
        self.children=[]  #collection of Children of that state
        self.parent=parent

    def add_child(self,child):
        self.children.append(child)  #child appending

In [ ]:
def child_addition(node):  #if an empty position is found , creating a possible new child
    current_state=node.state.copy()
    current_player=node.player
    present_children_count=len(node.children)
    Num_empty_positions=0
    for i in range(3):
        for j in range(3):
            if current_state[i,j]==0:
                Num_empty_positions+=1
                if Num_empty_positions>present_children_count:
                    current_state[i,j]=current_player
                    new_child=Tic_Tac_Toe_Board(state=current_state,player=-1*current_player,parent=node)
                    node.add_child(child=new_child)   # added new child
                    return new_child


In [ ]:
#Selection of the best child of a node using ucb estimate
def UCB_selection(node):
    children_of_node=node.children
    best_posible_child=children_of_node[0]
    UCB_estimate=-100000 #keeps track of ucb estimate
    for child in children_of_node:
        current_player=child.player
        action_value=current_player*child.q
        explore=np.sqrt(np.log(node.n)/child.n)
        if action_value+explore>UCB_estimate:
            best_posible_child=child
            UCB_estimate=action_value+explore
    return best_posible_child



In [ ]:
#one or more nodes(childs) are created , the subtree at the root is expanded
def Expansion(root_of_subtree):
    node=root_of_subtree
    while node.childs<=len(node.children):
        if (node.state==0).sum()==0:
            return node
        node=UCB_selection(node) #best node according to ucb estimate
    if node.childs>node.n:
        node=child_addition(node)
    return node

In [ ]:

def current_status_of_game(state):
  #checking the possibility of win of any player along row/column / diagonal
  #return of 1 if player of symbol 1 has won
  #return of -1 if player of symbol -1 has won
  #return of 0 if match draw
  #return of 2 if game still continues
    for i in range(3):
        if np.sum(state[:,i])==3:
            return 1
        elif np.sum(state[:,i])==-3:
            return -1
        if np.sum(state[i,:])==3:
            return 1
        elif np.sum(state[i,:])==-3:
            return -1
    if state[0,0]+state[1,1]+state[2,2]==3:
        return 1
    elif state[0,0]+state[1,1]+state[2,2]==-3:
        return -1
    if state[0,2]+state[1,1]+state[2,0]==3:
        return 1
    elif state[0,2]+state[1,1]+state[2,0]==-3:
        return -1
    if (state==0).sum()==0:
        return 0
    return 2



In [ ]:
def play_random_move(state,player):
    #Player plays randomly
    current_state=state.copy()
    row_index,col_index=random.randint(0,2),random.randint(0,2)
    while current_state[row_index,col_index]!=0:  #stops only if an empty position is found at random
        row_index,col_index=random.randint(0,2),random.randint(0,2)
    current_state[row_index,col_index]=player  #state updated
    return current_state

In [ ]:
#Playing of game (randomly)
#One complete game is played
def play_out(node):
    current_state=node.state
    current_player=node.player
    while current_status_of_game(current_state)==2:  #game continues till the end(win by either of the players or draw)
        current_state=play_random_move(current_state,current_player) # a random move is played by the player
        current_player*=-1 # opponent player plays the next play
    return current_status_of_game(current_state) #indicates which player won or draw

In [ ]:
#the simulation result is propagated back until root node is met
def Back_propagation(leaf,result):
    node_current=leaf
    while node_current!=None:
        node_current.n+=1
        if result==1:
            node_current.w+=1
        elif result==-1:
            node_current.l+=1
        node_current.q=(node_current.w-node_current.l)/node_current.n
        node_current=node_current.parent # node updated to parent of that node

In [ ]:
def best_child(node): #finding the best child of a node
    children=node.children
    best_child=children[0]
    action_value_of_best_child=-1000  #-1000 is the worst possible action value possible
    for child in children:
        if child.q>action_value_of_best_child:
            best_child=child
            action_value_of_best_child=child.q
    return best_child


In [ ]:
#implementing Monte Carlo Tree Search
#num= Number of games for taining
def Monte_Carlo_Tree_Search(root,num):
    for i in range(num):
        leaf=Expansion(root)
        simulation_result=play_out(leaf)
        Back_propagation(leaf,simulation_result)
    return best_child(root)


**B)**

In [ ]:
#A suitable board configuration in which the MCTS agent(symbol "1") is one move away from win
Board_position=np.array([[1,0,-1],[0,1,-1],[0,0,0]])
print("Board before Play:\n",Board_position)
node=Tic_Tac_Toe_Board(state= Board_position,player=1,parent=None)
Board_After_Play=Monte_Carlo_Tree_Search(node,1000)
print("Board after Play:\n",Board_After_Play.state)

Board before Play:
 [[ 1  0 -1]
 [ 0  1 -1]
 [ 0  0  0]]
Board after Play:
 [[ 1  0 -1]
 [ 0  1 -1]
 [ 0  0  1]]


In [ ]:
# A suitable board configuration in which the MCTS agent(symbol "1") is one move away from loss
Board_position= np.array([[-1,1,1],[0,-1,-1],[-1,1,0]])
print("Board before Play:\n",Board_position)
node=Tic_Tac_Toe_Board(state= Board_position,player=1,parent=None)
Board_After_Play=Monte_Carlo_Tree_Search(node,1000)
print("Board after Play:\n",Board_After_Play.state)

Board before Play:
 [[-1  1  1]
 [ 0 -1 -1]
 [-1  1  0]]
Board after Play:
 [[-1  1  1]
 [ 1 -1 -1]
 [-1  1  0]]


In [ ]:
#The board configuration where the opponent has made the first move and has occupied the centre square
Board_position= np.array([[0,0,0],[0,-1,0],[0,0,0]])
print("Board before Play:\n",Board_position)
node=Tic_Tac_Toe_Board(state= Board_position,player=1,parent=None)
Board_After_Play=Monte_Carlo_Tree_Search(node,1000)
print("Board after Play:\n",Board_After_Play.state)

Board before Play:
 [[ 0  0  0]
 [ 0 -1  0]
 [ 0  0  0]]
Board after Play:
 [[ 0  0  1]
 [ 0 -1  0]
 [ 0  0  0]]


**C)**

In [ ]:
#Opponent is randomly playing
def random_opponent_agent(board_state):
    row_index,col_index=random.randint(0,2),random.randint(0,2)
    while board_state[row_index,col_index]!=0: #stops only if an empty position is found at random
        row_index,col_index=random.randint(0,2),random.randint(0,2)
    board_state[row_index,col_index]=-1 #opponent symbol "-1"
    return board_state


In [ ]:
#Opponent is safely playing
#opponent symbol "-1"
def safe_opponent_agent(board_state):
    for i in range(3):
        if board_state[i][0]+board_state[i][1]+board_state[i][2]==-2 or board_state[i][0]+board_state[i][1]+board_state[i][2]==2:
            if (board_state[i][0]==-1 and board_state[i][1]==-1) or (board_state[i][0]==1 and board_state[i][1]==1):
                board_state[i,2]=-1
                return board_state
            elif (board_state[i][0]==-1 and board_state[i][2]==-1) or (board_state[i][0]==1 and board_state[i][2]==1):
                board_state[i,1]=-1
                return board_state
            else:
                board_state[i,0]=-1
                return board_state
        elif board_state[0][i]+board_state[1][i]+board_state[2][i]==-2 or board_state[0][i]+board_state[1][i]+board_state[2][i]==2:
            if (board_state[0][i]==-1 and board_state[1][i]==-1) or (board_state[0][i]==1 and board_state[1][i]==1):
                board_state[2,i]=-1
                return board_state
            elif (board_state[0][i]==-1 and board_state[2][i]==-1) or (board_state[0][i]==1 and board_state[2][i]==1):
                board_state[1,i]=-1
                return board_state
            else:
                board_state[0,i]=-1
                return board_state
    if board_state[0][0]+board_state[1][1]+board_state[2][2]==-2 or board_state[0][0]+board_state[1][1]+board_state[2][2]==2:
        if (board_state[0][0]==-1 and board_state[1][1]==-1) or (board_state[0][0]==1 and board_state[1][1]==1):
            board_state[2,2]=-1
            return board_state
        elif (board_state[0][0]==-1 and board_state[2][2]==-1) or (board_state[0][0]==1 and board_state[2][2]==1):
            board_state[1,1]=-1
            return board_state
        else:
            board_state[0,0]=-1
            return board_state
    return random_opponent_agent(board_state)

In [ ]:

def play(N,agent,Playing_position_first):
  #Playing_position_first = True , then MCTS agent plays first
    wins=0
    losses=0
    for i in tqdm(range(N),desc="Loading:"):
        state=np.zeros((3,3))
        if Playing_position_first==True:
            state=Monte_Carlo_Tree_Search(Tic_Tac_Toe_Board(state=state,player=1,parent=None),1000).state
        while current_status_of_game(state)==2:
            state=agent(state)
            if current_status_of_game(state)==2:
                state=Monte_Carlo_Tree_Search(Tic_Tac_Toe_Board(state=state,player=1,parent=None),1000).state
            else:
                break
        if current_status_of_game(state)==1:
            wins+=1
        elif current_status_of_game(state)==-1:
            losses+=1
    return wins,losses, (1000-wins-losses)


In [ ]:
#against random player
wins,losses,draws=play(1000,random_opponent_agent,True)
print("Playing against random player:")
print("wins:",wins,"  losses:",losses," draws:",draws)

Loading:: 100%|██████████| 1000/1000 [35:49<00:00,  2.15s/it]

Playing against random player:
wins: 973   losses: 12  draws: 15


In [ ]:
#against safe player
wins,losses,draws=play(1000,safe_opponent_agent,True)
print("Playing against safe player:")
print("wins:",wins,"  losses:",losses," draws:",draws)

Loading:: 100%|██████████| 1000/1000 [39:56<00:00,  2.40s/it]

Playing against safe player:
wins: 653   losses: 68  draws: 279


**D)**

In [ ]:
def MCTS_vs_MCTS(N,Playing_position_first):
    #Playing_position_first = True , then MCTS agent plays first
    wins=0
    losses=0
    for i in tqdm(range(N),desc="Loading:"):
        state=np.zeros((3,3))
        if Playing_position_first==True:
            state=Monte_Carlo_Tree_Search(Tic_Tac_Toe_Board(state=state,player=1,parent=None),1000).state
        while current_status_of_game(state)==2:
            state*=-1
            state=Monte_Carlo_Tree_Search(Tic_Tac_Toe_Board(state=state,player=1,parent=None),1000).state
            state*=-1
            if current_status_of_game(state)==2:
                state=Monte_Carlo_Tree_Search(Tic_Tac_Toe_Board(state=state,player=1,parent=None),1000).state
            else:
                break
        if current_status_of_game(state)==1:
            wins+=1
        elif current_status_of_game(state)==-1:
            losses+=1
    return wins,losses, (1000-wins-losses)

In [ ]:
#against mcts playing first
wins,losses,draws=MCTS_vs_MCTS(1000,True)
print("Playing (first move )against MCTS player :")
print("wins:",wins,"  losses:",losses," draws:",draws)

Loading:: 100%|██████████| 1000/1000 [1:03:29<00:00,  3.81s/it]

Playing (first move )against MCTS player :
wins: 617   losses: 135  draws: 248


In [ ]:
#against mcts playing second
wins,losses,draws=MCTS_vs_MCTS(1000,False)
print("Playing (second move )against MCTS player:")
print("wins:",wins,"  losses:",losses," draws:",draws)

Loading:: 100%|██████████| 1000/1000 [59:08<00:00,  3.55s/it]

Playing (second move )against MCTS player:
wins: 175   losses: 590  draws: 235


From the above results, we know for sure that the first player has a higher chance of winning when both the players use MCTS strategy.